### Processa Decisões
1. Identifica o resultado (Improcedente, Procedente ou Procedente em Parte)
2. Atualiza os processos com as infos da decisão (Data, Resultado e Juiz)


In [ ]:
# Importando as Bibliotecas
import pandas as pd
import json
import pymongo
import yaml

In [ ]:
# Carrega Configurações do arquivo config
with open("../config.yaml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)

user = cfg["mongodb"]["user"]
passwd = cfg["mongodb"]["passwd"]
host = cfg["mongodb"]["host"]
dbmongo = cfg["mongodb"]["db"]

In [ ]:
# Cria Conexão com banco
url_conexao = "mongodb+srv://" + user + ":" + passwd + "@" + host + "/?retryWrites=true&w=majority"
client = pymongo.MongoClient(url_conexao)
db = client[dbmongo]


In [ ]:
# Coleções Mongo
col_decisoes = db['decisoes']

# Decisões

In [ ]:
# Carrega base de decisões total
dados_decisoes = pd.DataFrame(list(col_decisoes.find({})))
df_decisoes = pd.json_normalize(dados_decisoes['decisao'])
df_decisoes.shape

In [ ]:
# Quantas decisões sem "resultado"
col_decisoes.count_documents({"decisao.resultado": {"$exists": False} })

### Tags "Parcialmente Procedente" 

In [ ]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( PARCIALMENTE PROCEDENTE)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Parcialmente Procedente'}}
)


col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( Parcialmente Procedente)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Parcialmente Procedente'}}
)

### "Tag Improcedente"

In [ ]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( IMPROCEDENTE)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Improcedente'}}
)

col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( IMPPROCEDENTE)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Improcedente'}}
)

col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( Improcedente)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Improcedente'}}
)

In [ ]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( improcedente)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Improcedente'}}
)

### Tag "Procedente"

In [ ]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( PROCEDENTE)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Procedente'}}
)

In [ ]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( Procedente)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Procedente'}}
)

In [ ]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( PROCEDENTE)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Procedente'}}
)

### Acordo

In [ ]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( acordo celebrado entre as partes )"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Acordo'}}
)

In [ ]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( acordo que chegaram as partes )"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Acordo'}}
)

In [ ]:
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( acordo a que chegaram as partes)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Acordo'}}
)

In [ ]:
#o acordo ora celebrado entre as partes
col_decisoes.update_many(
    {'$and': [
        {'decisao.texto_decisao':{'$regex':"( acordo a que chegaram as partes)"}},
        {"decisao.resultado": {"$exists": False}}
    ]},
    {'$set':{'decisao.resultado':'Acordo'}}
)

In [ ]:
col_decisoes.update_many(
    {'$and':[
        {'decisao.texto_decisao': {'$regex':"( HOMOLOGO )"}},
        {'decisao.texto_decisao': {'$regex':"( JULGO EXTINTO )"}},
        {"decisao.resultado": {"$exists": False} }]},
    {'$set':{'decisao.resultado':'Acordo'}}
)

### Extinto

In [ ]:
col_decisoes.update_many(
    {'$and':[
        {'decisao.texto_decisao': {'$regex':"( JULGO EXTINT)"}},
        {"decisao.resultado": {"$exists": False} }]},
    {'$set':{'decisao.resultado':'Extinto'}}
)

In [ ]:
col_decisoes.update_many(
    {'$and':[
        {'decisao.texto_decisao': {'$regex':"( EXTINTA a presente ação)"}},
        {"decisao.resultado": {"$exists": False} }]},
    {'$set':{'decisao.resultado':'Extinto'}}
)